# IBM-Coursera Data Science Capstone Project - Week 3

## Section 1: Web scraping and creating a dataframe

In [9]:
!pip install folium

     |████████████████████████████████| 93 kB 2.8 MB/s eta 0:00:011


In [10]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import folium
%matplotlib inline
from sklearn.cluster import KMeans

print("Libraries Imported")

Libraries Imported


In [13]:
data_url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [14]:
tables = pd.read_html(data_url) 
df = tables[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [15]:
#clean data, drop Borough="Not assigned"
df = df[df.Borough != "Not assigned"]
df = df.reset_index(drop=True)

In [16]:
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [17]:
df.shape

(103, 3)

## Section 2: Applying lat long to Neighborhoods

In [19]:
link = "http://cocl.us/Geospatial_data"
df2 = pd.read_csv(link)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
df3 = pd.merge(df, df2, on="Postal Code")
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Section 3: Cluster neighborhoods

In [26]:
map = folium.Map(location=[43.651070,-79.347015],zoom_start=10, height=900)
for lat,lng,borough,neighbourhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map)
map

In [27]:
CLIENT_ID = 'FKNYLWC40IUCVAZ3W0R4NCEWZLBJ0LETV2Z0BOKISGS2TF0W' # your Foursquare ID
CLIENT_SECRET = 'QCUT0EUIEUB5FGCAFIPSFNGBTRNVIZDWPWMAKMHG0IMNDXOQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FKNYLWC40IUCVAZ3W0R4NCEWZLBJ0LETV2Z0BOKISGS2TF0W
CLIENT_SECRET:QCUT0EUIEUB5FGCAFIPSFNGBTRNVIZDWPWMAKMHG0IMNDXOQ


In [40]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
import requests
toronto_venues = getNearbyVenues(names = df3['Neighbourhood'],
                                   latitudes = df3['Latitude'],
                                   longitudes = df3['Longitude']
                                  )

Parkwoods


KeyError: 'groups'

In [43]:
toronto_venues.head()

NameError: name 'toronto_venues' is not defined

## ^^ Not sure what's going on here -- how do I fix a 'KeyError'